<a href="https://colab.research.google.com/github/Tlen33/Recurrent-neural-networks-and-one-dimensional-convolutional-networks-for-word-processing/blob/main/%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B5_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_%D0%B8_%D0%BE%D0%B4%D0%BD%D0%BE%D0%BC%D0%B5%D1%80%D0%BD%D1%8B%D0%B5_%D1%81%D0%B2%D1%91%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_%D0%B4%D0%BB%D1%8F_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B8_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт библиотек

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Embedding, Flatten, Activation
from tensorflow.keras.layers import Conv1D, SpatialDropout1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from google.colab import files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

%matplotlib inline

In [ ]:
# Подключаем диск

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Чтение файлов в текст

def readText(fileName):
  f = open(fileName, 'r')
  text = f.read()
  text = text.replace("\n", " ")
  
  return text

trainText = []
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Аппендицит.txt'))
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Гастрит.txt'))
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Гепатит.txt'))
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Дуоденит.txt'))
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Колит.txt'))
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Панкреатит.txt'))
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Холицестит.txt'))
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Эзофагит.txt'))
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Энтерит.txt'))
trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни/Язва.txt'))

className = ["Аппендицит", "Гастрит", "Гепатит", "Дуоденит", "Колит", "Панкреатит", "Холицестит", "Эзофагит", "Энтерит", "Язва"]
nClasses = len(className)

testText = []
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Аппендицит_тест.txt'))
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Гастрит_тест.txt'))
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Гепатит_тест.txt'))
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Дуоденит_тест.txt'))
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Колит_тест.txt'))
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Панкреатит_тест.txt'))
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Холицестит_тест.txt'))
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Эзофагит_тест.txt'))
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Энтерит_тест.txt'))
testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Болезни_тест/Язва_тест.txt'))

In [ ]:
# Преобразовываем текстовые данные в числовые/векторные для обучения нейросети

maxWordsCount = 20000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='–—!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\xa0–\ufeff', lower=True, split=' ', char_level=False, oov_token = 'unknown')
tokenizer.fit_on_texts(trainText)
items = list(tokenizer.word_index.items())
print(items[:10])

[('unknown', 1), ('в', 2), ('и', 3), ('боль', 4), ('тошнота', 5), ('или', 6), ('рвота', 7), ('боли', 8), ('живота', 9), ('при', 10)]


In [ ]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю

trainWordIndexes = tokenizer.texts_to_sequences(trainText)
testWordIndexes = tokenizer.texts_to_sequences(testText)

In [ ]:
# Формирование обучающей выборки по листу индексов слов

def getSetFromIndexes(wordIndexes, xLen, step):
  xSample = []
  wordsLen = len(wordIndexes)
  index = 0
  while (index + xLen <= wordsLen):
    xSample.append(wordIndexes[index:index+xLen])
    index += step
    
  return xSample

In [ ]:
# Формирование обучающей и проверочной выборки

def createSetsMultiClasses(wordIndexes, xLen, step):
  nClasses = len(wordIndexes)
  classesXSamples = []
  for wI in wordIndexes:
    classesXSamples.append(getSetFromIndexes(wI, xLen, step))

  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)):
      xSamples.append(xT[i])
    
    currY = utils.to_categorical(t, nClasses)
    for i in range(len(xT)):
      ySamples.append(currY)

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [ ]:
# Формируем обучающую и тестовую выборку

xLen = 50
step = 5
xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

In [ ]:
# Представляем тестовую выборку в удобных для распознавания размерах

def createTestMultiClasses(wordIndexes, xLen, step):
  nClasses = len(wordIndexes)
  xTestClasses01 = []
  xTestClasses = []
  for wI in wordIndexes:
    sample = (getSetFromIndexes(wI, xLen, step))
    xTestClasses.append(np.array(sample))
    xTestClasses01.append(tokenizer.sequences_to_matrix(sample))
  xTestClasses01 = np.array(xTestClasses01)
  
  return xTestClasses01, xTestClasses

def recognizeMultiClass(model, xTest, modelName):
  print("НЕЙРОНКА: ", modelName)
  print()
  
  val = []
  pred = []
  totalSumRec = []
  
  for i in range(nClasses):
    currPred = model.predict(xTest[i])
    pred.append(currPred)
    currOut = np.argmax(currPred, axis=1)
    evVal = []
    sumRec = []
    for j in range(nClasses):
      sumRec.append(0)
      for t in range(len(currOut)):
        if (currOut[t] == j):
          sumRec[j] += 1
      evVal.append(sumRec[j] / len(currOut))
    totalSumRec.append(sumRec[i])
    
    val.append(evVal[i])
    recognizedClass = np.argmax(evVal)
    
    isRecognized = "распознано НЕ ВЕРНО!"
    if (recognizedClass == i):
      isRecognized = "распознано ВЕРНО!"
    print(className[i], " распознано ", int(100*evVal[i]), "% сеть считает, что это ", className[recognizedClass], ", ", isRecognized)
  
  print()
  sumVal = 0
  sumCount = 0
  lenXtest = []
  for i in range(nClasses):
    sumCount += len(xTest[i])
    sumVal += totalSumRec[i]
  print("Средний процент распознавания ", int(100*sumVal/sumCount), "%")
  return sumVal/sumCount

xTestClasses01, xTestClasses = createTestMultiClasses(testWordIndexes, xLen, step)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# Нейронка Embedding + LSTM

model1 = Sequential()
model1.add(Embedding(maxWordsCount, 10, input_length=xLen))
model1.add(SpatialDropout1D(0.2))
model1.add(BatchNormalization())
model1.add(LSTM(4))
model1.add(Dense(10, activation='softmax'))
model1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history = model1.fit(xTrain, yTrain, epochs=100, batch_size=128, validation_data=(xTest, yTest))

Epoch 1/100
13/13 [==============================] - 3s 62ms/step - loss: 2.2806 - accuracy: 0.1515 - val_loss: 2.2904 - val_accuracy: 0.1573
Epoch 2/100
13/13 [==============================] - 0s 25ms/step - loss: 2.2195 - accuracy: 0.2641 - val_loss: 2.2826 - val_accuracy: 0.2000
Epoch 3/100
13/13 [==============================] - 0s 22ms/step - loss: 2.1711 - accuracy: 0.3793 - val_loss: 2.2744 - val_accuracy: 0.2160
Epoch 4/100
13/13 [==============================] - 0s 24ms/step - loss: 2.1212 - accuracy: 0.4265 - val_loss: 2.2641 - val_accuracy: 0.2560
Epoch 5/100
13/13 [==============================] - 0s 22ms/step - loss: 2.0759 - accuracy: 0.4841 - val_loss: 2.2525 - val_accuracy: 0.2160
Epoch 6/100
13/13 [==============================] - 0s 22ms/step - loss: 2.0289 - accuracy: 0.4997 - val_loss: 2.2387 - val_accuracy: 0.2373
Epoch 7/100
13/13 [==============================] - 0s 22ms/step - loss: 1.9863 - accuracy: 0.5204 - val_loss: 2.2244 - val_accuracy: 0.3093
Epoch 

In [ ]:
pred = recognizeMultiClass(model1, xTestClasses, "Embedding + LSTM")

НЕЙРОНКА:  Embedding + LSTM

Аппендицит  распознано  60 % сеть считает, что это  Аппендицит ,  распознано ВЕРНО!
Гастрит  распознано  95 % сеть считает, что это  Гастрит ,  распознано ВЕРНО!
Гепатит  распознано  90 % сеть считает, что это  Гепатит ,  распознано ВЕРНО!
Дуоденит  распознано  100 % сеть считает, что это  Дуоденит ,  распознано ВЕРНО!
Колит  распознано  92 % сеть считает, что это  Колит ,  распознано ВЕРНО!
Панкреатит  распознано  100 % сеть считает, что это  Панкреатит ,  распознано ВЕРНО!
Холицестит  распознано  94 % сеть считает, что это  Холицестит ,  распознано ВЕРНО!
Эзофагит  распознано  90 % сеть считает, что это  Эзофагит ,  распознано ВЕРНО!
Энтерит  распознано  91 % сеть считает, что это  Энтерит ,  распознано ВЕРНО!
Язва  распознано  94 % сеть считает, что это  Язва ,  распознано ВЕРНО!

Средний процент распознавания  92 %


In [ ]:
# Нейронка Embedding + Conv1D

model2 = Sequential()
model2.add(Embedding(maxWordsCount, 10, input_length=xLen))
model2.add(SpatialDropout1D(0.2))
model2.add(BatchNormalization())
model2.add(Conv1D(20, 5, activation="relu", padding='same'))
model2.add(Conv1D(20, 5, activation="relu"))
model2.add(MaxPooling1D(2))
model2.add(Dropout(0.2))
model2.add(BatchNormalization())
model2.add(Flatten())
model2.add(Dense(10, activation='softmax'))
model2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history = model2.fit(xTrain, yTrain, epochs=100, batch_size=100, validation_data=(xTest, yTest))

Epoch 1/100
16/16 [==============================] - 2s 27ms/step - loss: 2.8688 - accuracy: 0.1191 - val_loss: 2.2816 - val_accuracy: 0.1840
Epoch 2/100
16/16 [==============================] - 0s 15ms/step - loss: 2.2092 - accuracy: 0.2511 - val_loss: 2.2667 - val_accuracy: 0.1840
Epoch 3/100
16/16 [==============================] - 0s 15ms/step - loss: 1.7109 - accuracy: 0.4252 - val_loss: 2.2734 - val_accuracy: 0.1840
Epoch 4/100
16/16 [==============================] - 0s 15ms/step - loss: 1.1434 - accuracy: 0.6311 - val_loss: 2.3234 - val_accuracy: 0.1840
Epoch 5/100
16/16 [==============================] - 0s 16ms/step - loss: 0.7393 - accuracy: 0.7638 - val_loss: 2.4192 - val_accuracy: 0.1840
Epoch 6/100
16/16 [==============================] - 0s 15ms/step - loss: 0.4567 - accuracy: 0.8770 - val_loss: 2.5700 - val_accuracy: 0.1840
Epoch 7/100
16/16 [==============================] - 0s 15ms/step - loss: 0.3037 - accuracy: 0.9243 - val_loss: 2.7086 - val_accuracy: 0.1840
Epoch 

In [ ]:
pred = recognizeMultiClass(model2, xTestClasses, "Embedding + Conv1D")

НЕЙРОНКА:  Embedding + Conv1D

Аппендицит  распознано  100 % сеть считает, что это  Аппендицит ,  распознано ВЕРНО!
Гастрит  распознано  100 % сеть считает, что это  Гастрит ,  распознано ВЕРНО!
Гепатит  распознано  96 % сеть считает, что это  Гепатит ,  распознано ВЕРНО!
Дуоденит  распознано  63 % сеть считает, что это  Дуоденит ,  распознано ВЕРНО!
Колит  распознано  100 % сеть считает, что это  Колит ,  распознано ВЕРНО!
Панкреатит  распознано  100 % сеть считает, что это  Панкреатит ,  распознано ВЕРНО!
Холицестит  распознано  81 % сеть считает, что это  Холицестит ,  распознано ВЕРНО!
Эзофагит  распознано  100 % сеть считает, что это  Эзофагит ,  распознано ВЕРНО!
Энтерит  распознано  60 % сеть считает, что это  Энтерит ,  распознано ВЕРНО!
Язва  распознано  100 % сеть считает, что это  Язва ,  распознано ВЕРНО!

Средний процент распознавания  92 %


In [ ]:
# Нейронка Embedding + Dense, LSTM и Conv1D

model3 = Sequential()
model3.add(Embedding(maxWordsCount, 10, input_length=xLen))
model3.add(SpatialDropout1D(0.2))
model3.add(LSTM(4, return_sequences=1))
model3.add(Dense(300, activation='relu'))
model3.add(Conv1D(20, 5, activation="relu")) 
model3.add(LSTM(4, return_sequences=1))
model3.add(Dropout(0.2))
model3.add(BatchNormalization())
model3.add(Conv1D(20, 5, activation="relu"))
model3.add(Conv1D(20, 5, activation="relu"))
model3.add(MaxPooling1D(2))
model3.add(Dropout(0.2))
model3.add(BatchNormalization())
model3.add(Flatten())
model3.add(Dense(10, activation='softmax'))
model3.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history = model3.fit(xTrain, yTrain, epochs=100, batch_size=128, validation_data=(xTest, yTest))

Epoch 1/100
13/13 [==============================] - 7s 182ms/step - loss: 2.4233 - accuracy: 0.1036 - val_loss: 2.2914 - val_accuracy: 0.1840
Epoch 2/100
13/13 [==============================] - 2s 116ms/step - loss: 2.3366 - accuracy: 0.1359 - val_loss: 2.2809 - val_accuracy: 0.1840
Epoch 3/100
13/13 [==============================] - 1s 114ms/step - loss: 2.2465 - accuracy: 0.1586 - val_loss: 2.2692 - val_accuracy: 0.1840
Epoch 4/100
13/13 [==============================] - 1s 113ms/step - loss: 1.9644 - accuracy: 0.2913 - val_loss: 2.2546 - val_accuracy: 0.1840
Epoch 5/100
13/13 [==============================] - 1s 115ms/step - loss: 1.6852 - accuracy: 0.3599 - val_loss: 2.2508 - val_accuracy: 0.1973
Epoch 6/100
13/13 [==============================] - 1s 113ms/step - loss: 1.4916 - accuracy: 0.4188 - val_loss: 2.2569 - val_accuracy: 0.0933
Epoch 7/100
13/13 [==============================] - 1s 113ms/step - loss: 1.3306 - accuracy: 0.4848 - val_loss: 2.2581 - val_accuracy: 0.0933

In [ ]:
pred = recognizeMultiClass(model3, xTestClasses, "Embedding + Dense, LSTM и Conv1D")

НЕЙРОНКА:  Embedding + Dense, LSTM и Conv1D

Аппендицит  распознано  100 % сеть считает, что это  Аппендицит ,  распознано ВЕРНО!
Гастрит  распознано  100 % сеть считает, что это  Гастрит ,  распознано ВЕРНО!
Гепатит  распознано  100 % сеть считает, что это  Гепатит ,  распознано ВЕРНО!
Дуоденит  распознано  100 % сеть считает, что это  Дуоденит ,  распознано ВЕРНО!
Колит  распознано  100 % сеть считает, что это  Колит ,  распознано ВЕРНО!
Панкреатит  распознано  95 % сеть считает, что это  Панкреатит ,  распознано ВЕРНО!
Холицестит  распознано  81 % сеть считает, что это  Холицестит ,  распознано ВЕРНО!
Эзофагит  распознано  100 % сеть считает, что это  Эзофагит ,  распознано ВЕРНО!
Энтерит  распознано  85 % сеть считает, что это  Энтерит ,  распознано ВЕРНО!
Язва  распознано  100 % сеть считает, что это  Язва ,  распознано ВЕРНО!

Средний процент распознавания  96 %
